Copyright (c) 2019 SK T-Brain

   Licensed under the Apache License, Version 2.0 (the "License");  
   you may not use this file except in compliance with the License.  
   You may obtain a copy of the License at  

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
   See the License for the specific language governing permissions and
   limitations under the License.

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 754 kB 8.8 MB/s 
     |████████████████████████████████| 3.0 MB 51.3 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.1
    Uninstalling transformers-4.9.1:
      Successfully uninstalled transformers-4.9.1


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_rqjik0s
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_rqjik0s


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [8]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-08-06 10:35:17--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-08-06 10:35:17--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc815a3df42d15367b77cbdebd53.dl.dropboxusercontent.com/cd/0/get/BTvGMT3_f5rirXsxMf2frbqce2XhdgwABv2echCNF7npKZ5UCuB0XdP9BsuKfaCYtcw7JvwZDZDA4fOredgJJbm3WEL5S5Dg2E5DBoQUTrX_sr-oFwLDjLdSft9V0dTYaxicQKiHseaQou4GeXu2-6sr/file?dl=1# [following]
--2021-08-06 10:35:17--  https://uc815a3df42d15367b77cbdebd53.dl.dropboxusercontent.com/cd/0/get/BTvGMT3_f5rirXsxMf2frbqce2XhdgwABv2echCNF7npKZ5UCuB0XdP9BsuKfaCYtcw7JvwZD

In [9]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

[[KoBERT 에러] TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str 해결 방법](https://tech-diary.tistory.com/27)

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6803467273712158 train acc 0.59375
epoch 1 batch id 201 loss 0.45077353715896606 train acc 0.5628109452736318
epoch 1 batch id 401 loss 0.44240233302116394 train acc 0.6780314837905237
epoch 1 batch id 601 loss 0.4050297141075134 train acc 0.7314891846921797
epoch 1 batch id 801 loss 0.4400184154510498 train acc 0.7607092696629213
epoch 1 batch id 1001 loss 0.35643500089645386 train acc 0.7791427322677322
epoch 1 batch id 1201 loss 0.33182936906814575 train acc 0.7920743130724396
epoch 1 batch id 1401 loss 0.4165557324886322 train acc 0.8012468772305497
epoch 1 batch id 1601 loss 0.3514646291732788 train acc 0.8091329637726421
epoch 1 batch id 1801 loss 0.24938030540943146 train acc 0.8154150471960022
epoch 1 batch id 2001 loss 0.3395545482635498 train acc 0.8211831584207896
epoch 1 batch id 2201 loss 0.27445343136787415 train acc 0.8263360404361654

epoch 1 train acc 0.829715941410694


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.88443094629156


...


epoch 3 test acc 0.8966791879795396


epoch 4 batch id 1 loss 0.37135282158851624 train acc 0.84375
epoch 4 batch id 201 loss 0.04200103133916855 train acc 0.9554570895522388
epoch 4 batch id 401 loss 0.1039196103811264 train acc 0.9572552992518704
epoch 4 batch id 601 loss 0.16814202070236206 train acc 0.9579346921797005
epoch 4 batch id 801 loss 0.21555009484291077 train acc 0.9590550873907615
epoch 4 batch id 1001 loss 0.20121797919273376 train acc 0.9604458041958042
epoch 4 batch id 1201 loss 0.099307119846344 train acc 0.9618026644462948
epoch 4 batch id 1401 loss 0.03579391539096832 train acc 0.9626382940756603
epoch 4 batch id 1601 loss 0.18443866074085236 train acc 0.9632065896314803
epoch 4 batch id 1801 loss 0.047233760356903076 train acc 0.9640217240421988
epoch 4 batch id 2001 loss 0.09731587022542953 train acc 0.9649315967016492
epoch 4 batch id 2201 loss 0.18196792900562286 train acc 0.9655909245797365

epoch 4 train acc 0.9661213914960182



epoch 4 test acc 0.898377557544757


epoch 5 batch id 1 loss 0.37574246525764465 train acc 0.90625
epoch 5 batch id 201 loss 0.030976301059126854 train acc 0.9745802238805971
epoch 5 batch id 401 loss 0.1282498985528946 train acc 0.9764650872817955
epoch 5 batch id 601 loss 0.1386735588312149 train acc 0.9758995424292846
epoch 5 batch id 801 loss 0.09588997066020966 train acc 0.9758504993757803
epoch 5 batch id 1001 loss 0.20704391598701477 train acc 0.9765859140859141
epoch 5 batch id 1201 loss 0.02281656488776207 train acc 0.9768812447960034
epoch 5 batch id 1401 loss 0.017043184489011765 train acc 0.976902658815132
epoch 5 batch id 1601 loss 0.1341361701488495 train acc 0.9770553560274828
epoch 5 batch id 1801 loss 0.010146687738597393 train acc 0.9774430871737924
epoch 5 batch id 2001 loss 0.015424462966620922 train acc 0.9777923538230885
epoch 5 batch id 2201 loss 0.03847269341349602 train acc 0.9780781462971376

epoch 5 train acc 0.978348976109215



epoch 5 test acc 0.897878037084399
